In [1]:
import enum
import time

import json
import requests
import os
from bs4 import BeautifulSoup
from retry import retry
import openai
import tiktoken
import pandas as pd
from settings import OPEN_AI_TOKEN
from llm.chat import ChatModel, OpenAIChatModel
from parsing.catalog import CatalogParser
from parsing.details import DetailsPageParser
from parsing.pagination import PaginationDetection
from scraping import SimpleScrapingFlow
from browser.local import LocalBrowserScraper

openai.api_key = OPEN_AI_TOKEN

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
scraper = LocalBrowserScraper()

2023-10-24 11:09:34,319 - WDM - INFO - ====== WebDriver manager ======
2023-10-24 11:09:35,436 - WDM - INFO - Get LATEST chromedriver version for google-chrome
2023-10-24 11:09:35,627 - WDM - INFO - Get LATEST chromedriver version for google-chrome
2023-10-24 11:09:35,781 - WDM - INFO - Get LATEST chromedriver version for google-chrome
2023-10-24 11:09:36,086 - WDM - INFO - WebDriver version 118.0.5993.70 selected
2023-10-24 11:09:36,090 - WDM - INFO - Modern chrome version https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/118.0.5993.70/mac-x64/chromedriver-mac-x64.zip
2023-10-24 11:09:36,091 - WDM - INFO - About to download new driver from https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/118.0.5993.70/mac-x64/chromedriver-mac-x64.zip
2023-10-24 11:09:36,274 - WDM - INFO - Driver downloading response is 200
2023-10-24 11:09:36,870 - WDM - INFO - Get LATEST chromedriver version for google-chrome
2023-10-24 11:09:37,233 - WDM - INFO - Driver has been saved in cache [

In [69]:
url = 'https://navigator.sk.ru/orn/1125636'

In [44]:
scraper.get(url)
time.sleep(1)
html = scraper.page_source

In [58]:
from lxml import etree

xpath = "//span[@class='description-block__title']"
soup = BeautifulSoup(scraper.page_source, features='lxml')
dom = etree.HTML(str(soup))
dom.xpath(xpath)[1].text

'Оценка зрелости компании'

In [62]:
parser = DetailsPageParser()
text = parser.to_selectors(html)
text

2023-10-17 22:14:41,631 - /Users/iakov/Documents/PycharmProjects/ScraperAI/backend/utils/compressor.py - INFO - Initial HTML length: 179319
2023-10-17 22:14:41,716 - /Users/iakov/Documents/PycharmProjects/ScraperAI/backend/utils/compressor.py - INFO - Final html length: 21659
2023-10-17 22:14:42,010 - /Users/iakov/Documents/PycharmProjects/ScraperAI/backend/utils/compressor.py - INFO - Initial HTML length: 179319
2023-10-17 22:14:42,096 - /Users/iakov/Documents/PycharmProjects/ScraperAI/backend/utils/compressor.py - INFO - Final html length: 21659


{'Subtitle': "//p[@class='page__subtitle']",
 'Participant since': "//div[@class='company-info-list__item']/div[@class='company-info-li__text'][1]",
 'Intellectual Property': "//div[@class='company-info-list__item']/a",
 'Year of Foundation': "//div[@class='company-info-block__item'][1]/div[@class='company-info-block__item-text-3']",
 'Revenue for 2022': "//div[@class='company-info-block__item'][2]/div[@class='company-info-block__item-text-3']/span",
 'Director': "//div[@class='sidebar-item contact-alpha-icon']/div[@class='information']/div[@class='information__subtitle']",
 'Address': "//a[@class='sidebar-item location-alpha-icon']/div[@class='information']/div[@class='information__title']",
 'Activity Field': "//div[@class='sidebar-item portfolio-alpha-icon activity-field']/div[@class='activity-field__information information']/span[@class='information__subtitle_link']",
 'Main OKVED': "//div[@class='sidebar-item documents-alpha-icon main-okved']/div[@class='main-okved__information in

In [37]:
flow = SimpleScrapingFlow()

In [63]:
df = flow.collect_all_data('https://navigator.sk.ru')
df

2023-10-17 22:16:46,502 - /Users/iakov/Documents/PycharmProjects/ScraperAI/backend/scraping/__init__.py - INFO - Start parsing "https://navigator.sk.ru"
2023-10-17 22:16:46,503 - WDM - INFO - ====== WebDriver manager ======
2023-10-17 22:16:46,874 - WDM - INFO - Get LATEST chromedriver version for google-chrome
2023-10-17 22:16:47,169 - WDM - INFO - Get LATEST chromedriver version for google-chrome
2023-10-17 22:16:47,456 - WDM - INFO - Driver [/Users/iakov/.wdm/drivers/chromedriver/mac64/118.0.5993.70/chromedriver-mac-x64/chromedriver] found in cache
2023-10-17 22:16:58,677 - /Users/iakov/Documents/PycharmProjects/ScraperAI/backend/utils/compressor.py - INFO - Initial HTML length: 540308
2023-10-17 22:16:58,805 - /Users/iakov/Documents/PycharmProjects/ScraperAI/backend/utils/compressor.py - INFO - Final html length: 22520
2023-10-17 22:17:00,986 - /Users/iakov/Documents/PycharmProjects/ScraperAI/backend/scraping/__init__.py - INFO - Pagination xpath was found: "//*[@class='load-more__

Response load-more__button w-button


2023-10-17 22:17:01,351 - /Users/iakov/Documents/PycharmProjects/ScraperAI/backend/utils/compressor.py - INFO - Initial HTML length: 540308
2023-10-17 22:17:01,481 - /Users/iakov/Documents/PycharmProjects/ScraperAI/backend/utils/compressor.py - INFO - Final html length: 22520
2023-10-17 22:17:01,746 - openai - INFO - error_code=rate_limit_exceeded error_message='Rate limit reached for gpt-4 in organization org-EzOvk9i9IuuTkezjlcsW6rXP on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-10-17 22:17:01,747 - retry.api - WARNING - Rate limit reached for gpt-4 in organization org-EzOvk9i9IuuTkezjlcsW6rXP on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues., retrying in 5 seconds...
2023-10-17 22:17:07,02

,Subtitle,Participant since,Intellectual Property,Year of Foundation,Revenue for 2022,Director,Address,Activity Field,Main OKVED,Project Manager,Description,Company Maturity Assessment,CRL (company readiness level),IRL (investment readiness level)
0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""НАУЧ...",Участник «Сколково»,Интеллектуальная собственность,2021,628 000,,,Производство электронного и электрического обо...,None,None,"ООО «НПП «Новотех» отечественный стартап, орга...",Компания ведет свою деятельность в регионах,4,7
1,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""САТОРИ""",Участник «Сколково»,Интеллектуальная собственность,2018,25 221 000,,,None,None,None,Платформа автоматизирует весь жизненный цикл у...,Компания ведет свою деятельность в регионах,None,None
2,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ЮРИДИ...,Участник «Сколково»,Интеллектуальная собственность,2023,None,,,None,None,None,None,Компания ведет свою деятельность в регионах,None,None
3,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЕКОМ...",Участник «Сколково»,None,2021,76 000,,,None,None,None,None,Компания ведет свою деятельность в регионах,None,None
4,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АТАМАН""",Участник «Сколково»,None,2023,None,,,None,None,None,РОИЗВОДСТВО ПНЕВМАТИЧЕСКОГО ОРУЖИЯ ATAMAN НАЧИ...,Компания ведет свою деятельность в регионах,None,None
5,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""КУША...",None,Интеллектуальная собственность,2020,94 813 000,,,None,None,None,None,Компания ведет свою деятельность в регионах,None,None
6,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ИКСС...",Участник «Сколково»,None,2021,246 197 000,,,None,None,None,None,Компания ведет свою деятельность в регионах,None,None
7,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ШЭНЭ...",None,None,2018,None,,,None,None,None,РАНЕВОЕ ПОКРЫТИЕ NOVOSKIN,Компания ведет свою деятельность в регионах,None,None
8,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ФРЕШ...",Участник «Сколково»,None,2023,None,,,None,None,None,None,Компания ведет свою деятельность в регионах,None,None
9,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""МЕГА...",Участник «Сколково»,None,2013,None,,,None,None,None,TONNA — PaaS-платформа бесшовных трансграничны...,Компания ведет свою деятельность в регионах,4,6


In [64]:
df.to_excel('test_output.xlsx', index=False)

In [4]:
flow = SimpleScrapingFlow()
df = flow.collect_all_data('https://маркет.промыслы.рф/goods-company/view/30')
df

2023-10-24 11:10:43,179 - /Users/iakov/Documents/PycharmProjects/ScraperAI/backend/scraping/__init__.py - INFO - Start parsing "https://маркет.промыслы.рф/goods-company/view/30"
2023-10-24 11:10:43,181 - WDM - INFO - ====== WebDriver manager ======
2023-10-24 11:10:43,570 - WDM - INFO - Get LATEST chromedriver version for google-chrome
2023-10-24 11:10:43,730 - WDM - INFO - Get LATEST chromedriver version for google-chrome
2023-10-24 11:10:43,871 - WDM - INFO - Driver [/Users/iakov/.wdm/drivers/chromedriver/mac64/118.0.5993.70/chromedriver-mac-x64/chromedriver] found in cache
2023-10-24 11:10:52,721 - /Users/iakov/Documents/PycharmProjects/ScraperAI/backend/utils/compressor.py - INFO - Initial HTML length: 47880
2023-10-24 11:10:52,789 - /Users/iakov/Documents/PycharmProjects/ScraperAI/backend/utils/compressor.py - INFO - Final html length: 11583
2023-10-24 11:10:57,080 - /Users/iakov/Documents/PycharmProjects/ScraperAI/backend/scraping/__init__.py - INFO - Pagination xpath was found: 

Response ajax-pagination-more


2023-10-24 11:10:57,317 - /Users/iakov/Documents/PycharmProjects/ScraperAI/backend/utils/compressor.py - INFO - Final html length: 11583
2023-10-24 11:11:01,019 - /Users/iakov/Documents/PycharmProjects/ScraperAI/backend/scraping/__init__.py - INFO - Found url classname: "addToCart"
2023-10-24 11:11:01,104 - /Users/iakov/Documents/PycharmProjects/ScraperAI/backend/scraping/__init__.py - INFO - Page: 0: Found 12 new urls
2023-10-24 11:11:04,285 - /Users/iakov/Documents/PycharmProjects/ScraperAI/backend/scraping/__init__.py - INFO - Page: 1: Found 24 new urls
2023-10-24 11:11:07,452 - /Users/iakov/Documents/PycharmProjects/ScraperAI/backend/scraping/__init__.py - INFO - Page: 2: Found 36 new urls
2023-10-24 11:11:10,513 - /Users/iakov/Documents/PycharmProjects/ScraperAI/backend/scraping/__init__.py - INFO - Totally found 36 urls
  0%|          | 0/10 [00:00<?, ?it/s]2023-10-24 11:11:12,487 - /Users/iakov/Documents/PycharmProjects/ScraperAI/backend/utils/compressor.py - INFO - Initial HTML

,Seller,Price,Description,Characteristic 1,Value 1,Characteristic 2,Value 2,Characteristic 3,Value 3,Characteristic 4,Value 4,Characteristic 5,Value 5,Characteristic 6,Value 6,url
0,Кубачинский художественный комбинат,43 600 ₽,Набор из серебра 875 пробы ручной работы Кубач...,Длина: ..........................................,"6,5 см",Проба: ..........................................,875,Материал: .......................................,"Серебро, чернь",Высота: .........................................,"13,5 см",Объем: ..........................................,140 мл,Вес: ............................................,220 g,https://маркет.промыслы.рф/goods/view/316287
1,Кубачинский художественный комбинат,143 000 ₽,Кувшин из серебра 875 пробы ручной работы Куба...,Длина: ..........................................,"10,5 см",Проба: ..........................................,875,Материал: .......................................,"Серебро, чернь",Высота: .........................................,37 см,Объем: ..........................................,1000 мл,Вес: ............................................,800 g,https://маркет.промыслы.рф/goods/view/281438
2,Кубачинский художественный комбинат,141 000 ₽,Набор из серебра 925 пробы ручной работы Кубач...,Проба: ..........................................,925,Материал: .......................................,"Серебро, чернь",Объем: ..........................................,225 мл,Вес: ............................................,780 g,None,None,None,None,https://маркет.промыслы.рф/goods/view/286153
3,Кубачинский художественный комбинат,12 800 ₽,Зажигалка из серебра 875 пробы ручной работы К...,Длина: ..........................................,"3,5 см",Проба: ..........................................,875,Материал: .......................................,"Серебро, чернь",Высота: .........................................,6 см,Вес: ............................................,45 g,None,None,https://маркет.промыслы.рф/goods/view/302520
4,Кубачинский художественный комбинат,3 000 ₽,None,Проба: ..........................................,925,Материал: .......................................,Серебро,Вес: ............................................,15 g,None,None,None,None,None,None,https://маркет.промыслы.рф/goods/view/316283
5,Кубачинский художественный комбинат,26 500 ₽,Подстаканники из серебра 875 пробы ручной рабо...,Диаметр: ........................................,7 см,Длина: ..........................................,"10,5 см",Проба: ..........................................,875,Материал: .......................................,"Серебро, чернь, позолота",Высота: .........................................,"9,5 см",Объем: ..........................................,250 мл,https://маркет.промыслы.рф/goods/view/284874
6,Кубачинский художественный комбинат,1 600 ₽,Браслет из латуни ручной работы Кубачинских ма...,Материал: .......................................,Латунь,Высота: .........................................,"3,8 см",None,None,None,None,None,None,None,None,https://маркет.промыслы.рф/goods/view/283049
7,Кубачинский художественный комбинат,40 000 ₽,Стакан из серебра 999 пробы ручной работы Куба...,Диаметр: ........................................,8 см,Проба: ..........................................,999,Материал: .......................................,Серебро,Высота: .........................................,9 см,Объем: ..........................................,380 мл,Вес: ............................................,175 g,https://маркет.промыслы.рф/goods/view/281970
8,Кубачинский художественный комбинат,36 000 ₽,Турка из серебра 875 пробы ручной работы Кубач...,Диаметр: ........................................,8 см,Проба: ..........................................,875,Материал: .......................................,"Серебро, чернь",Высота: .........................................,11 см,Объем: ..........................................,250 мл,Вес: ..............................